In [359]:
import numpy as np
import pandas as pd
import sys

# TO DO
# implement: substituion(), consider_Strategy()
# fix guards for team abbreviations in command line call
# make sure team abbreviations match for both of the 2 df's
# update file path for 2 df's once we move code to a server on cs
# actually adjust the fatigue adjustment function in the loop of the main program at the bottom
# possibly adjust probability of scoring or time between scores if game scores are unrealistic
# implement a "baseline" strategy to compete against our agent and lose


class State():
    quarter = 1
    
    time_in_qtr = 12.00
    home_team = [] # needs to be a df probably with their: minutes in game, fatigue-adjusted-plusminus, "real plus/minus" & maybe Names
    away_team = []
    home_players_on = [] # this can just be a list of the 5 players who the coach has put in the game... maybe use their names for convenience
    away_players_on = [] 
    home_strategy = "Normal"
    away_strategy = "Normal"
    home_score = 0
    away_score = 0
    momentum = 0 # lets say positive means home has scored last, negative means away scored last
    momentum_queue = 0 # so we can track maybe the last 5 baskets or so... rather than just streak
    possession_of_ball = "Neither" # or "Away" or "Home"
    home_timeouts_left = 6 # is this correct for nba? i think
    away_timeouts_left = 6

def consider_timeout(team, state):
    probability_of_calling_timeout = 0.50 # we can totally arbitrarily redefine this, so that it makes sense such that teams don't...
    # ... waste all their timeouts early and so that they dont save them all up too long... but it's mainly based on momentum
    if state.possession_of_ball == team:
        if team == "Home":
              if state.home_timeouts_left > 0:
                    if state.momentum < -5:
                        # away team has momentum
                        u = np.random.uniform()
                        if u > probability_of_calling_timeout:
                            return True
    elif team == "Away":
        if state.away_timeouts_left > 0:
            if state.momentum > 5:
              # home team has momentum
                u = np.random.uniform()
                if u > probability_of_calling_timeout:
                    return True
    return False

def donothing():
    return 'donothing'
    # literally do nothing
def simulate_points(state):
      # account for who has ball, FAPM of both teams, time in game, strategies, momentum
    
    ##
    ##
    
    home_fapm = sum(state.home_team[state.home_team.NAME.isin(state.home_players_on.NAME)]['FAPM'])
    away_fapm = sum(state.away_team[state.away_team.NAME.isin(state.away_players_on.NAME)]['FAPM'])
    
    '''
    
    try to find you the best setting for value of "u > 0.50 + home_fapm - away_fapm"
    
    '''
    if(state.possession_of_ball == "Home"):
        u = np.random.uniform()
        #print('Home team Turns')
        if(u >  0.7 - (away_fapm - home_fapm)/away_fapm):
          # 0 points - turnover
            #print('donothing')
            donothing()
        else:
            #print('try to shoot the ball')
            u = np.random.uniform()
            if(u < 2/5):
                state.home_score += 2
                #print('2 points get.')
            elif(2/5 < u < 0.1+(2/5)):
                state.home_score += 3
                #print('3 points get.')
            else:
                state.home_score += 1
                #print('1 points get.')
        #print('--------------------------------------------------------')
        state.possession_of_ball = "Away"
        
    elif(state.possession_of_ball == "Away"):
        u = np.random.uniform()
        #print('Away team Turns')
        if(u > 0.7 -(away_fapm - home_fapm)/away_fapm):
            # 0 points - turnover
            #print('donothing')
            donothing()
        else:
            u = np.random.uniform()
            #print('try to shoot the ball')
            if(u < 2/5):
                state.away_score += 2
                #print('2 points get.')
            elif(2/5 < u < 0.1+(2/5)):
                state.away_score += 3
                #print('3 points get.')
            else:
                state.away_score += 1
                #print('1 points get.')
        #print('--------------------------------------------------------')
        state.possession_of_ball = "Home"
        
def substitution(team, state):
    if team == "Home":
        tmp = state.home_players_on
        state.home_players_on = state.home_team.sort_values(by=['FAPM'], ascending=False).iloc[0:5]
        if len(set(tmp) - set(state.home_players_on)) > 0 and len(set(state.home_players_on) - set(tmp)) > 0:
            print(', '.join(set(tmp) - set(state.home_players_on)) + " is substituted by " + ', '.join(set(state.home_players_on) - set(tmp)))
    

    elif team == "Away":
        tmp = state.away_players_on
        state.away_players_on = state.away_team.sort_values(by=['FAPM'], ascending=False).iloc[0:5]
        if len(set(tmp) - set(state.away_players_on)) > 0 and len(set(state.away_players_on) - set(tmp)) > 0:
            print(', '.join(set(tmp) - set(state.away_players_on)) + " is substituted by " + ', '.join(set(state.away_players_on) - set(tmp)))
            
            
            
def simulate_time(state, home_avg, away_avg):
    if(state.possession_of_ball == "Home"):
        u = np.random.exponential(1/home_avg)[0]
    elif(state.possession_of_ball == "Away"):
        u = np.random.exponential(1/away_avg)[0]
        # for all players on court, update minutes played
    state.home_players_on['Minutes in Game'] = 0
    state.away_players_on['Minutes in Game'] = 0
    
    state.home_players_on['Minutes in Game'] = state.home_players_on['Minutes in Game'].add(u) # for every row in the column
    state.away_players_on['Minutes in Game'] = state.away_players_on['Minutes in Game'].add(u) # for every row in the column
    if(state.time_in_qtr - u < 0):
        state.quarter += 1
        state.time_in_qtr = 12.00
        if(first_poss == "Home"):
            if(state.quarter%2 == 0):
                state.possession_of_ball = "Away"
            else:
                state.possession_of_ball = "Home"
        elif(first_poss == "Away"):
            if(state.quarter%2 != 0):
                state.possession_of_ball = "Away"
            else:
                state.possession_of_ball = "Home"
        return False # false means do not simulate points
    else:
        state.time_in_qtr -= u
        return True  
    
def update_time(state):
    state.home_team.loc[state.home_team.NAME.isin(state.home_players_on.NAME), 'Minutes in Game'] += state.home_players_on['Minutes in Game']
    state.away_team.loc[state.away_team.NAME.isin(state.away_players_on.NAME), 'Minutes in Game'] += state.away_players_on['Minutes in Game']
    
def update_fapm(state):
    
    #discuss
    state.home_team.loc[state.home_team.NAME.isin(state.home_players_on.NAME), 'FAPM'] -= 0.005
    state.away_team.loc[state.away_team.NAME.isin(state.away_players_on.NAME), 'FAPM'] -= 0.005   
    
    state.home_team.loc[~state.home_team.NAME.isin(state.home_players_on.NAME), 'FAPM'] += 0.005
    state.away_team.loc[~state.away_team.NAME.isin(state.away_players_on.NAME), 'FAPM'] += 0.005

In [360]:
state = State()

all_players = pd.read_csv("NBA_RPM.csv")
home_tm = 'ATL'
temp = all_players[all_players['TEAM'] == home_tm].sort_values(by=['MPG'], ascending=False)[0:10]

home = {}
home['NAME'] = temp['NAME'] 
home['RPM'] = temp['RPM'] 
home['Minutes in Game'] = 0
home['FAPM'] = temp['RPM']

state.home_team = []
state.home_team.append(home)
state.home_team = pd.DataFrame.from_dict(state.home_team[0], orient='columns')

away_tm = 'BOS'
temp = all_players[all_players['TEAM'] == away_tm].sort_values(by=['MPG'], ascending=False)[0:10]

away = {}
away['NAME'] = temp['NAME'] 
away['RPM'] = temp['RPM'] 
away['Minutes in Game'] = 0
away['FAPM'] = temp['RPM']

state.away_team = []
state.away_team.append(away)
state.away_team = pd.DataFrame.from_dict(state.away_team[0], orient='columns')

poss_times = pd.read_csv("NBA_Poss.csv")
home_tm_poss_secs = poss_times[poss_times['TEAM'] == home_tm]['SPP']
away_tm_poss_secs = poss_times[poss_times['TEAM'] == away_tm]['SPP']

u = np.random.uniform()
if(u < 0.5):
    state.possession_of_ball = "Home"
    first_poss = "Home"
else:
    state.possession_of_ball = "Away"
    first_poss = "Away"
    
count = state.quarter

while(state.quarter < 5):
    
    consider_timeout("Home", state)
    consider_timeout("Away", state)
    substitution("Home", state)
    substitution("Away", state)
    #consider_strategy("Home", state)
    #consider_strategy("Away", state)
    
    try_score = simulate_time(state, home_tm_poss_secs, away_tm_poss_secs)

    if(try_score):
        simulate_points(state)
        
    update_time(state)    
    update_fapm(state)
    
    if state.quarter == (count + 1):
        print('--------------------------------------------------')
        print('Quarter ' + str(count) + ' end')
        print(home_tm,': ', state.home_score, ' | ', away_tm,': ', state.away_score)
        count = state.quarter

--------------------------------------------------
Quarter 1 end
ATL :  46  |  BOS :  53
--------------------------------------------------
Quarter 2 end
ATL :  73  |  BOS :  82
--------------------------------------------------
Quarter 3 end
ATL :  96  |  BOS :  103
--------------------------------------------------
Quarter 4 end
ATL :  103  |  BOS :  117


In [364]:
state.home_team

,NAME,RPM,Minutes in Game,FAPM
6,Trae Young,6.95,48.708691,3.435
64,John Collins,3.00,46.810555,-0.165
226,De'Andre Hunter,-1.71,14.523006,-0.155
119,Kevin Huerter,0.91,29.791410,-0.165
107,Bogdan Bogdanovic,1.64,35.942899,-0.165
17,Clint Capela,5.38,48.708691,1.865
544,Wes Iwundu,-7.39,0.000000,-3.875
190,Danilo Gallinari,-1.14,19.058204,-0.165
378,Cameron Oliver,-3.83,0.000000,-0.315
500,Chaundee Brown Jr.,-5.54,0.000000,-2.025


In [363]:
state.away_team

,NAME,RPM,Minutes in Game,FAPM
3,Jayson Tatum,8.96,48.708691,5.445
19,Jaylen Brown,5.16,46.390481,2.005
67,Marcus Smart,2.97,30.946072,2.005
24,Robert Williams III,4.82,43.617961,2.005
92,Derrick White,2.05,24.657014,2.005
14,Al Horford,5.94,48.708691,2.425
352,Grant Williams,-3.54,0.000000,-0.025
205,Daniel Theis,-1.46,0.514545,2.005
411,Kelan Martin,-4.11,0.000000,-0.595
266,Payton Pritchard,-2.42,0.000000,1.095
